In [2]:
import requests
from bs4 import BeautifulSoup
from math import ceil, nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

In [3]:
url = "https://www.carrefour.es/supermercado/berlinas-bombon-donuts-4-ud/R-715505824/p"
page = render_html(url)

Carrefour.es
Supermercado
Productos Frescos
Panadería, Bollería y Pastelería
Magdalenas, Dulces y Bollos


In [62]:
def find_ean(page):
    for script in page.find_all("script",{"type":"text/javascript","charset":"utf-8"}):
        strt = r"var dataLayer \= window\.dataLayer \|\| \[\]\;dataLayer\.push\("
        end = r"\)\;"
        text = re.search(f"(?<={strt})(.*?)(?={end})",script.get_text())
        if text != None:
            text = json.loads(text.group())
            product = {
                "code": text['productEAN'],
                "brand": text['productBrand'],
                "name": text['productName']
            }
            return product

def find_image_url(page):
    script = page.find("script",{"type":"application/ld+json"})
    script = json.loads(script.get_text())
    return script['image']

def find_calories(page):
    info_container = page.find("div",class_="nutrition")
    
    # find ingredients
    ingredients = info_container.find("p",class_="nutrition-ingredients__content").get_text().strip()
    
    # find calories
    cal = info_container.find("p",class_="nutrition-graph__kcal").get_text().strip().lower()
    netto = info_container.find("p",class_="nutrition-graph__info-text").get_text().strip().lower()
    netto = re.search("\d+ [a-z\/]+",netto)
    calories = None    
    if netto != None:
        calories = f"{cal} / {netto.group()}"
        
    # find nutriscore
    nutriscore = info_container.find("div",class_="nutrition__score").get_text().strip()
    return calories, ingredients

image_url = find_image_url(page)
calories, ingredients = find_calories(page)

categories = []
for li in page.find("ul",class_="breadcrumb__list").find_all("li"):
    categories.append(li.get_text().strip())

product_data = {
    "code" : product['code'],
    "sources" : ["src"],
    "image_url" : image_url,
    "small_image_url" : image_url,
    "product_name" : product['name'],
    "brand" : product['brand'],
    "nutriscore" : None,
    "categories" : categories,
    "ingredients_text" : ingredients,
    "calories" : calories
}

product_data

{'code': ['8410022109674', '8410022110472'],
 'sources': ['src'],
 'image_url': 'https://static.carrefour.es/hd_510x_/img_pim_food/488506_00_1.jpg',
 'small_image_url': 'https://static.carrefour.es/hd_510x_/img_pim_food/488506_00_1.jpg',
 'product_name': 'berlinas-bombon-donuts-4-ud',
 'brand': 'donuts',
 'nutriscore': None,
 'categories': ['Carrefour.es',
  'Supermercado',
  'Productos Frescos',
  'Panadería, Bollería y Pastelería',
  'Magdalenas, Dulces y Bollos'],
 'ingredients_text': 'Harina de trigo, grasa vegetal de palma, agua, azúcar, grasa vegetal totalmente hidrogenada de palmiste, cacao desgrasado en polvo (3,8%), levadura, lactosa, jarabe de glucosa y fructosa, huevo líquido pasteurizado, emulgentes (E 471, lecitina de girasol, E 492, E 481, E 472e), sal, dextrosa, gluten de trigo, harina de soja, almidón, leche desnatada en polvo, estabilizantes (E 412, E 341i), conservador (E 200), aromas y agente de tratamiento de la harina (E 300). Puede contener avellana.',
 'calories'